# Spark SQL

Структура шага

* аудио 07_sql
    * по нему слайды "Что такое Spark SQL", "Как использовать", "Какие операции поддерживаются", "Использование SQL и Structured API", "Logical plan", "Кэширование таблиц"
* остальное - материалы

### Что такое Spark SQL

* свой собственный SQL движок
* может использоваться "вперемешку" с датафреймами
* технически может работать (и работает) с Hive Metastore
* поддерживает подмножество ANSI SQL:2003
* есть интерактивная консоль (`spark-sql`)

### Как использовать

* основной метод - `sql()` (метод объекта SparkSession)
* возвращает dataframe
* lazy evaluation
* может использоваться для доступа к источникам
    * не уверен в параллелизме
* хорошо работает с Hive (эффективно, параллельно)

### Какие операции поддерживаются

* создание таблиц и view
    * включая внешние (поддерживается синтаксис Hive QL)
* создание баз
* `insert` 
* основное - `select`

### Использование SQL и Structured API

* dataframe можно "зарегистрировать" как view (`.createOrReplaceTempView("some_sql_view")`)
* все трансформации - lazy
* в итоге все будет скомпилировано в операции над RDD


### Logical plan

* трансформации формируют логический план (алгоритм)
* его можно посмотреть (`.explain()`)
* планы запросов SQL и dataframe эквивалентны

### Кэширование таблиц

* `CACHE TABLE` - кэширует таблицу
* `UNCACHE TABLE` - чистит кэш

### Попрактикуемся

Для простоты будем все делать локально - зарегистрируем таблицу со странами, как `view` и будем с ней работать с помощью SQL.

In [1]:
import os
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
os.environ["SPARK_HOME"] = "/home/mk/mk_win/projects/SparkEdu/lib/python3.5/site-packages/pyspark"
os.environ["PYSPARK_PYTHON"] = "/usr/bin/python3"
os.environ["PYSPARK_DRIVER_PYTHON"] = "python3"
os.environ["PYSPARK_SUBMIT_ARGS"] = "pyspark-shell"

In [2]:
spark = SparkSession.builder.master("local").appName("spark_sql").getOrCreate()

In [6]:
cdf = spark.read.format("csv") \
    .option("mode", "FAILFAST") \
    .option("inferSchema", "true") \
    .option("header","true") \
    .option("path", "data/countries of the world.csv") \
    .load()
cdf.createOrReplaceTempView("countries")

**Простой select**

In [4]:
spark.sql("select count(*) from countries").show()

+--------+
|count(1)|
+--------+
|     227|
+--------+



**Информация о таблице**

In [5]:
spark.sql("describe table countries").show()

+--------------------+---------+-------+
|            col_name|data_type|comment|
+--------------------+---------+-------+
|             Country|   string|   null|
|              Region|   string|   null|
|          Population|      int|   null|
|      Area (sq. mi.)|      int|   null|
|Pop. Density (per...|   string|   null|
|Coastline (coast/...|   string|   null|
|       Net migration|   string|   null|
|Infant mortality ...|   string|   null|
|  GDP ($ per capita)|      int|   null|
|        Literacy (%)|   string|   null|
|   Phones (per 1000)|   string|   null|
|          Arable (%)|   string|   null|
|           Crops (%)|   string|   null|
|           Other (%)|   string|   null|
|             Climate|   string|   null|
|           Birthrate|   string|   null|
|           Deathrate|   string|   null|
|         Agriculture|   string|   null|
|            Industry|   string|   null|
|             Service|   string|   null|
+--------------------+---------+-------+



**Что с пробелами**

Посмотрим - как спробелами в названиях стран и регионов

In [11]:
spark.sql("select distinct region,length(region) from countries").show(100,False)

+-----------------------------------+--------------+
|region                             |length(region)|
+-----------------------------------+--------------+
|EASTERN EUROPE                     |35            |
|OCEANIA                            |35            |
|SUB-SAHARAN AFRICA                 |35            |
|NORTHERN AMERICA                   |35            |
|NEAR EAST                          |35            |
|WESTERN EUROPE                     |35            |
|BALTICS                            |35            |
|ASIA (EX. NEAR EAST)               |29            |
|NORTHERN AFRICA                    |35            |
|C.W. OF IND. STATES                |20            |
|LATIN AMER. & CARIB                |23            |
+-----------------------------------+--------------+



**Поработаем смешанно - SQL + dataframe**

In [12]:
spark.sql("select country,region,population from countries") \
    .filter(f.col('country').startswith('Russia')) \
    .show()

+-------+--------------------+----------+
|country|              region|population|
+-------+--------------------+----------+
|Russia |C.W. OF IND. STATES | 142893540|
+-------+--------------------+----------+



**Logical plans**

Посмотрим на логические планы выполнения SQL запроса и dataframe и убедимся в их "похожести" (найти различия, конечно, можно, но они - косметические). 

In [21]:
sqlQ = spark.sql("select region,count(*) as ncountries from countries group by region order by ncountries desc")
dfQ = cdf.groupBy('Region').count().withColumnRenamed("count","ncountries").sort(f.desc('ncountries'))
dfQ.show()
sqlQ.show()
dfQ.explain()
sqlQ.explain()

+--------------------+----------+
|              Region|ncountries|
+--------------------+----------+
|SUB-SAHARAN AFRIC...|        51|
|LATIN AMER. & CAR...|        45|
|ASIA (EX. NEAR EA...|        28|
|WESTERN EUROPE   ...|        28|
|OCEANIA          ...|        21|
|NEAR EAST        ...|        16|
|EASTERN EUROPE   ...|        12|
|C.W. OF IND. STATES |        12|
|NORTHERN AFRICA  ...|         6|
|NORTHERN AMERICA ...|         5|
|BALTICS          ...|         3|
+--------------------+----------+

+--------------------+----------+
|              region|ncountries|
+--------------------+----------+
|SUB-SAHARAN AFRIC...|        51|
|LATIN AMER. & CAR...|        45|
|ASIA (EX. NEAR EA...|        28|
|WESTERN EUROPE   ...|        28|
|OCEANIA          ...|        21|
|NEAR EAST        ...|        16|
|EASTERN EUROPE   ...|        12|
|C.W. OF IND. STATES |        12|
|NORTHERN AFRICA  ...|         6|
|NORTHERN AMERICA ...|         5|
|BALTICS          ...|         3|
+------------

In [22]:
spark.stop()